Install Spark

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [0]:
!wget -q http://www.gtlib.gatech.edu/pub/apache/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz

In [0]:
!tar -xvf spark-2.4.5-bin-hadoop2.7.tgz

In [0]:
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [0]:
!pip install -q pyspark

In [0]:
import findspark
findspark.init("spark-2.4.5-bin-hadoop2.7") # SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
from pyspark import SparkContext
sc =SparkContext.getOrCreate()

In [0]:
# !spark-shell


Load data using python commands

In [0]:
myrdd = sc.textFile("/content/drive/My Drive/Amazon_Responded_Oct05.csv").map(lambda line: line.split(","))

In [0]:
type(myrdd)

pyspark.rdd.PipelinedRDD

In [0]:
myrdd.count()

505839

DataFrame

In [0]:
myrdd_df = spark.read.csv("/content/drive/My Drive/Amazon_Responded_Oct05.csv", header = True, inferSchema=True)

In [0]:
type(myrdd_df)

pyspark.sql.dataframe.DataFrame

In [0]:
myrdd_df.count()

413247

In [0]:
myrdd_df.show()

+--------------------+--------------------+----------------+-----------+-------------------+---------------------+--------------+-----------------+--------------+--------------------+-------------+-------------+--------------------+------------------+--------------------+--------------+--------------------+--------------+---------+-----------------------+-------------------------+-----------------------+-------------+---------+--------------------+
|              id_str|    tweet_created_at|user_screen_name|user_id_str|user_statuses_count|user_favourites_count|user_protected|user_listed_count|user_following|    user_description|user_location|user_verified|user_followers_count|user_friends_count|     user_created_at|tweet_language|               text_|favorite_count|favorited|in_reply_to_screen_name|in_reply_to_status_id_str|in_reply_to_user_id_str|retweet_count|retweeted|                text|
+--------------------+--------------------+----------------+-----------+-------------------+--

DF that only includes 6 columns

In [0]:
new_rdd = myrdd_df.select('id_str', "tweet_created_at", "user_verified", "favorite_count", "retweet_count", "text_" )

Remove records where “user_verified” is “FALSE”

In [0]:
new_myrdd_df = new_rdd.filter(myrdd_df.user_verified == "True")


In [0]:
new_myrdd_df.count()

171797

Group by created date

In [0]:
new_myrdd_df.show(5)

+--------------------+--------------------+-------------+--------------+-------------+--------------------+
|              id_str|    tweet_created_at|user_verified|favorite_count|retweet_count|               text_|
+--------------------+--------------------+-------------+--------------+-------------+--------------------+
|'793281386912354304'|Tue Nov 01 02:39:...|         True|             0|            0|@SeanEPanjab I'm ...|
|'793502854459879424'|Tue Nov 01 17:19:...|         True|             0|            0|@SeanEPanjab Plea...|
|'793504235400884224'|Tue Nov 01 17:25:...|         True|             0|            0|@SeanEPanjab With...|
|'793513446633533440'|Tue Nov 01 18:02:...|         True|             0|            0|@SeanEPanjab I'm ...|
|'793301295255945216'|Tue Nov 01 03:59:...|         True|             0|            0|@aakashwangnoo Hi...|
+--------------------+--------------------+-------------+--------------+-------------+--------------------+
only showing top 5 rows



In [0]:
df = new_myrdd_df
df2 = df.withColumn("day", df.tweet_created_at.substr(5,7))
df2.show()

+--------------------+--------------------+-------------+--------------+-------------+--------------------+-------+
|              id_str|    tweet_created_at|user_verified|favorite_count|retweet_count|               text_|    day|
+--------------------+--------------------+-------------+--------------+-------------+--------------------+-------+
|'793281386912354304'|Tue Nov 01 02:39:...|         True|             0|            0|@SeanEPanjab I'm ...|Nov 01 |
|'793502854459879424'|Tue Nov 01 17:19:...|         True|             0|            0|@SeanEPanjab Plea...|Nov 01 |
|'793504235400884224'|Tue Nov 01 17:25:...|         True|             0|            0|@SeanEPanjab With...|Nov 01 |
|'793513446633533440'|Tue Nov 01 18:02:...|         True|             0|            0|@SeanEPanjab I'm ...|Nov 01 |
|'793301295255945216'|Tue Nov 01 03:59:...|         True|             0|            0|@aakashwangnoo Hi...|Nov 01 |
|'793423313674571776'|Tue Nov 01 12:03:...|         True|             0|

In [0]:
df2_grouped= df2.groupby("day")

NameError: ignored

using sql


In [0]:
df2.createOrReplaceTempView("table1")

In [0]:
df2.show()

NameError: ignored

In [0]:
query = '''SELECT day, count(day) as sum FROM table1 GROUP BY day ORDER BY sum DESC'''

In [0]:
spark.sql(query).show(5)

+-------+----+
|    day| sum|
+-------+----+
|Jan 03 |1536|
|Jan 10 |1508|
|Jan 11 |1496|
|Jan 12 |1410|
|Jan 06 |1364|
+-------+----+
only showing top 5 rows



In [0]:
query1 = '''SELECT id_str, (favorite_count+ retweet_count) as total, text_  FROM table1 WHERE day = "Jan 03 " ORDER BY total DESC LIMIT 100'''

In [0]:
df4 = spark.sql(query1)

In [0]:
df4.show()

+--------------------+-----+--------------------+
|              id_str|total|               text_|
+--------------------+-----+--------------------+
|'816329761530093568'|  5.0|@amazon worst sho...|
|'816083406962434048'|  3.0|@ItsJosshA We alw...|
|'816086117938319360'|  2.0|@ItsJosshA Oh no!...|
|'816095108013654017'|  2.0|@KStefl Sounds li...|
|'816109446069911554'|  2.0|@Schoey1992 Happy...|
|'816157517428523008'|  2.0|@ratbones666 You ...|
|'816217909819297792'|  2.0|@ThorpPerrow Awww...|
|'816314295680110593'|  2.0|@thedexterouz Hi!...|
|'816090553595084800'|  1.0|@matt_linsley Ple...|
|'816165445925474304'|  1.0|@VlSlT Sorry to h...|
|'816323706431668226'|  1.0|@PPramod2041984 H...|
|'816190084546498560'|  1.0|@mailstosandeep H...|
|'816072943495249921'|  1.0|@Elidan_8 Here's ...|
|'816074813408165888'|  1.0|@joyfulneesh Than...|
|'816077071801810944'|  1.0|@brooklynnnross I...|
|'816086664053456896'|  1.0|@DurhamBelle I'm ...|
|'816092970168549376'|  1.0|@heypardeep We're...|


In [0]:
import pandas as pd

df5 = df4.select("text_").toPandas()
df5.to_csv("df5_text.txt")

In [0]:
counts = sc.textFile("df5_text.txt")\
.flatMap(lambda line: line.split()\
.map(lambda word: (word,1))\
.reduceByKey(lambda v1, v2: v1+v2)

In [0]:
word_freq = sorted(counts.collect(), key= lambda freq: freq[1], reverse=True)
df6 = spark.createDataFrame(word_freq, ["Word", "Frequency"])
df6.repartition(1).write.format("com.databricks.spark.csv").option("header", "true")\
.save("/content/drive/My Drive/ids561_hw2_solutions/word_freq.csv")
df6.show(5)

+----+---------+
|Word|Frequency|
+----+---------+
| the|       63|
|  to|       62|
| for|       50|
| you|       45|
|your|       32|
+----+---------+
only showing top 5 rows



The word with the highest frequency was 'the', which was repeated 63 times

Task 2

In [0]:
find_text_rdd = spark.read.csv("/content/drive/My Drive/find_text.csv", header = True, inferSchema=True)

In [0]:
find_text_rdd.show(5)

+--------------------+----+
|              id_str|text|
+--------------------+----+
|'793270689780203520'|null|
|'793281386912354304'|null|
|'793299404975247360'|null|
|'793301295255945216'|null|
|'793315815411978240'|null|
+--------------------+----+
only showing top 5 rows



In [0]:
find_text_rdd.createOrReplaceTempView("tablefind")

In [0]:
query_find = '''SELECT DISTINCT(table1.id_str) as id_str, text_ as text FROM table1 LEFT JOIN tablefind ON table1.id_str = tablefind.id_str '''

In [0]:
df7 = spark.sql(query_find)
df7.count()#.show(5)

171797

In [0]:
df7.repartition(1).write.format("com.databricks.spark.csv").option("header", "true")\
.save("/content/drive/My Drive/ids561_hw2_solutions/find_text.csv")